In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
df = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/tRNA_Phe_0724.xlsx')
# df_phe = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S2to5combined_20ul_40min.xlsx')
df = thermo_df(df)
df.shape
# plotly_zone(df_phe)

In [ ]:
df_total = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/total_tRNA_acid_deg.xlsx')
df_total = thermo_df(df_total)
df_total.shape

In [ ]:
df_total_top = df_total.sort_values('Vol', ascending=False).iloc[:1000]
df_total_top.to_excel('/Users/bryan/Downloads/top800.xlsx')

In [ ]:
plotly_zone(df_total)

In [ ]:
df_5p = pd.read_excel('/Users/bryan/Downloads/phe5p.xlsx')
df_3p = pd.read_excel('/Users/bryan/Downloads/phe3p.xlsx')
df_5p.shape, df_3p.shape

In [ ]:
df_5p = df_5p.sort_values('Vol', ascending=False).iloc[:1000]
df_3p = df_3p.sort_values('Vol', ascending=False).iloc[:1000]

In [ ]:
# masses = [24252.31, 24581.38]
# masses = [25334.57, 25005.52]
# masses = [24596.36, 24267.31]
# masses = [23904.24, 23599.19, 24233.28] # Asp
masses = [24287.35, 24460.38] # Ala, Val
# masses = [24460.38] # Val
# masses = [23545.17] # unknow 

In [ ]:
full_mass = masses[0]
df_pairs0 = computational_data_seperation(df_total, df_total, full_mass, ignore_endpoints=True)
full_mass = masses[1]
df_pairs1 = computational_data_seperation(df_total, df_total, full_mass, ignore_endpoints=True)
# plotly_zones(*df_ladders)
# print(df_pairs0[0].shape, df_pairs1[0].shape)

# df_tmp0 = df_pairs0[0].copy()

df_tmp0 = df_pairs0[0]#.sort_values('Vol', ascending=False).iloc[:20]
# print(df_tmp0.shape)
# df_tmp0 = df_tmp0.iloc[:30]
# print(df_tmp0.shape)
df_tmp0.to_excel('/Users/bryan/Downloads/ms0.xlsx')
# df_tmp1 = df_pairs1[0].copy()
df_tmp1 = df_pairs1[0]#.sort_values('Vol', ascending=False).iloc[:20]
df_tmp1.to_excel('/Users/bryan/Downloads/ms1.xlsx')
# plotly_zones(df_tmp0, df_tmp1)

In [ ]:
# df_tmp = df_pairs0[0].copy()
# df_tmp.sort_values('Vol', ascending=False).iloc[30]
plotly_zone(df_tmp0)
df_tmp0.shape, df_tmp1.shape

In [ ]:
idxs = list()
# create our callback function
def on_selection(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

def on_click(trace, points, selector):
#     print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

df_sample = df_tmp0
f = zone_selection(df_sample, on_selection=on_selection, on_click=on_click)
f

In [ ]:
df_chosen = df_sample.iloc[idxs]
df_chosen.shape

In [ ]:
df_mid_5p = df_chosen.copy()

In [ ]:
df_mid_3p = df_chosen.copy()

In [ ]:
df_mid = df_chosen.copy()
fullmass_dot = pd.DataFrame()
# df_gap = find_gap_dots(df_total, df_mid, fullmass_dot, major=True)
df_gap = find_gap_dots(df_total, df_mid_3p, fullmass_dot, major=True, orientation=3)
df_gap.shape

In [ ]:
plotly_zones(df_mid_3p, df_gap)

In [ ]:
df_asp = pd.concat([df_mid, df_gap]).drop_duplicates()
bcr = base_calling_random(df_asp)
plotly_basecalling(*bcr, annotate=False)

In [ ]:
df_tmp = df_pairs0[1].copy()
df_tmp.Mass += 2*M
df_ox = match_dfs(df_tmp, df)
plotly_zones(df_pairs0[0], df_ox)
# plotly_zone(df_ox)

In [ ]:
df1 = pd.read_csv('/Users/bryan/Downloads/phe2_75_5p.csv')
df2 = pd.read_csv('/Users/bryan/Downloads/phe2_75_3p.csv')
df3 = pd.read_csv('/Users/bryan/Downloads/phe2_76_5p.csv')
df4 = pd.read_csv('/Users/bryan/Downloads/phe2_76_3p.csv')
# df1[df1.Match==True].head()
# df2[df2.Match==True].head()
df1.head(), df3.head()

In [ ]:
df_total = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/total_tRNA_acid_deg.xlsx')
df_total = thermo_df(df_total)
# df_tyr.to_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/total_tRNA_acid_deg_std.xlsx')

In [ ]:

df_cca, df_cc = cds_isoforms(df_3p, df_3p, A)

In [ ]:
df_cca.to_excel('/Users/bryan/Downloads/cca.xlsx')

In [ ]:
df_3p_isoforms, df_3p_isoforms_i = cds_isoforms(df_3p, df_cc, 79.97)
# plotly_zones(df_3p_isoforms, df_3p_isoforms_i)

In [ ]:
def cds_3d(df3p, df5p, full_mass, error=0.1, ignore_endpoints=False, has_tag=False, tag=826.3184):
    df3p_mass_np = np.array(df3p['Mass'])
    df5p_mass_np = np.array(df5p['Mass'])
    mass_sum_np = df3p_mass_np[:, np.newaxis] + df5p_mass_np
    mass_sum_1 = np.round(mass_sum_np, 1)
    mass_sum_df = pd.DataFrame(mass_sum_np)
    mass_sum_df1 = pd.DataFrame(mass_sum_1)

    if has_tag:
        sum_value = round(full_mass + tag + 79.9663 - 18.0106, 1)
        print('full_mass {} sum_value {}'.format(full_mass, sum_value))
    else:
        sum_value = round(full_mass + 18.0106, 1)
        print('full_mass {} sum_value {}'.format(full_mass, sum_value))

    tmp = mass_sum_df1[(mass_sum_df1 >= sum_value-error) & (mass_sum_df1 <= sum_value+error)]
    final_idx_pairs = list(tmp[tmp.notnull()].stack().index)
    df3_idxs = [pair[0] for pair in final_idx_pairs]
    df5_idxs = [pair[1] for pair in final_idx_pairs]
    df3_idxs = list(set(df3_idxs))
    df5_idxs = list(set(df5_idxs))
#     print("df3 {} df5 {}".format(len(df3_idxs), len(df5_idxs)))

    df3p_selected = df3p.iloc[df3_idxs]
    df5p_selected = df5p.iloc[df5_idxs]
    
    if not ignore_endpoints:
        df5p_endpoints = get_end5p_points(df5p, full_mass + 18.0106, tag)
        print("df5p_endpoints {}".format(df5p_endpoints[['Mass', 'RT', 'Vol']]))

        df3p_selected.update(df5p_endpoints)
        df5p_selected.update(df5p_endpoints)
        
    return df3p_selected, df5p_selected

In [ ]:
# full_mass = 22782.14087 #20566.83957 #20911.88697 #21216.92827 #20261.79827 #19955.77297 #22437.09347 ##23472.23567 #21521.96957 #21827.01087 ## #
# full_mass = 24338.48264 #24643.52394 #24949.54924
# full_mass = 11238.5368 #23393.1479 #23088.1066
full_mass = 23506.2668 #23851.3142
# df_pairs0 = computational_data_seperation(df_5p, df, full_mass, ignore_endpoints=True)
df_pairs0 = computational_data_seperation(df, df_3p, full_mass, ignore_endpoints=True)
plotly_zones(*df_pairs0)
list(df_pairs0[0].sort_values('Mass').Mass), '---->', list(df_pairs0[1].sort_values('Mass').Mass)

In [ ]:
ms = 1358.194132 #2194.33335932565 #11319.4977708134 #10668.384177 #3149.507861 #1358.194132
df[(df.Mass>=ms) & (df.Mass<ms+0.1)]

# Match sequence with Mass shift

In [ ]:
# df = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/tRNA_Phe_0724.xlsx')
df_phe = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S2to5combined_20ul_40min.xlsx')
df = thermo_df(df)
df.shape

In [ ]:
df_phe2_5p = pd.read_excel('/Users/bryan/Studio/tools/phe_75_5p.xlsx')
df_phe2_5p.shape, df_phe2_5p.tail()

In [ ]:
df_tmp = df_phe2_5p.copy()
df_found1 = match_dfs(df_phe2_5p, df, ppm=10)
shift = 2*14
df_tmp.Mass += shift
df_found2 = match_dfs(df_tmp, df, ppm=10)
plotly_zones(df_found1, df_found2)

In [ ]:
df_matches = list()
# df_phe_match = match_dfs(df_phe2_5p, df)
# print(df_phe_match.shape)
for shift in range(0, 19, 1):
    df_tmp = df_phe2_5p.copy()
    df_tmp.Mass += shift
    df_found = match_dfs(df_tmp, df, ppm=15)
    if not df_found.empty:
        print(f'shift {shift}')
        df_matches.append((df_found, f'{shift}'))
#     plotly_zones(df_phe_match, df_found)

In [ ]:
dfs = list()
# dfs.append(df_phe_match)
# dfs.extend(df_matches)
# print(dfs)
dfs_chosen = [df_matches[0][0], df_matches[2][0], df_matches[16][0]] #, df_matches[1], df_matches[15], df_matches[17]
names = [df_matches[0][1], df_matches[2][1], df_matches[16][1]] #['base', '1', '14']#, '2', '16', '18'
plotly_multi_zones(dfs_chosen, names=names)
# plotly_multi_zones([x[0] for x in df_matches], names=[x[1] for x in df_matches])
# plt.scatter(df_phe_match.Mass, df_phe_match.RT)
# for idx, df in enumerate(df_matches):
#     print(idx)
#     if idx == 18:
#         print(df)
#     plt.scatter(df.Mass, df.RT, label=f'{idx}')
# plt.legend()

In [ ]:
for i in range(1, 50, 1):
    df_tmp = df_phe2_5p.copy()
    df_tmp.Mass += i
    df_found = match_dfs(df_tmp, df)
    print(i, df_found.shape)

In [ ]:
fullmass = 24642.532301 # #24642.532301
df_pairs = computational_data_seperation(df_5p, df_3p, fullmass, ignore_endpoints=True)
df_pairs[0].sort_values('Mass')

# Combinations of random 3 compounds

In [ ]:
from itertools import combinations

In [ ]:
dftop = pd.read_excel('/Users/bryan/Downloads/top800.xlsx')
dftop.shape

In [ ]:
dftop = dftop.iloc[:5]
list(dftop.iterrows())

In [ ]:
dftop['idx'] = dftop.index
dftop = dftop[['idx', 'Mass', 'RT', 'Vol']]
# dftop['idx'].astype(int)
# print(list(dftop.iterrows()))

In [ ]:
for row in dftop.iterrows()

In [ ]:
# def is_valid(data):
dftop.sort_values('Vol', ascending=False)
sums = list()
idxs = list()
for comb in combinations(dftop.iloc[:500].iterrows(), 3):
#     print(comb[0][1], comb[0][1][0], comb[0][1][1])
    masssum = sum([x[1][1] for x in comb]) - 18.0106 * 2
#     masssum = sum(comb) - 18.0106 * 2
    sums.append(masssum)
    idx = [int(x[1][0]) for x in comb]
    idxs.append(idx)
#     break

len(sums), len(idxs)

In [ ]:
df_sums = pd.Series(sums)
df_idxs = pd.DataFrame(idxs)

In [ ]:
df_sums.head(), df_idxs.head()

In [ ]:
pd_sums = pd.Series(sums)

In [ ]:
mass = 24280.3
len(pd_sums[(pd_sums>mass) & (pd_sums<mass+0.1)])

In [ ]:
# df_comm_5p = match_dfs(df_pairs0[0], df_pairs1[0])
# plotly_zone(df_comm_5p)

# df_tmp = df_pairs1[1].copy()
# df_tmp.Mass += A
# df_comm_3p = match_dfs(df_tmp, df_pairs0[1])
# plotly_zone(df_comm_3p)

plotly_zone(df_pairs0[0], y='RT')

# bcr = base_calling_random(df_pairs1[0])
# plotly_basecalling(*bcr, annotate=False)

In [ ]:
df_pairs1[0].sort_values('Mass', ascending=False)

In [ ]:
orientation = 3
df_sample = df_3p
mask = np.abs(df_sample.Mass - full_mass) < 0.1
fullmass_dot = df_sample[mask]
df_gap_phe76 = find_gap_dots(df_sample, df_3p_cc, fullmass_dot, major=True, orientation=orientation)
df_gap_phe76.shape

In [ ]:
dfa = df_3p_cc.copy()
dfb = df_gap_phe76.copy()
dfa['isSkeleton'] = True
dfb['isSkeleton'] = False
dfc = pd.concat([dfa, dfb])
plotly_zones(df_3p_cc, df_gap_phe76)
full_mass - dfc.Mass